In [59]:
import pandas as pd
idx = pd.IndexSlice

In [60]:
world_cases = pd.read_csv('data/world_daily_cases.csv',
        parse_dates=['date'],
        index_col=['country'],
        squeeze=True
        ).sort_index()

In [61]:
world_cases.head()

,date,cumulative_total_cases,daily_new_cases,active_cases,cumulative_total_deaths,daily_new_deaths
country,,,,,,
Afghanistan,2020-02-15,0.0,NaN,0.0,0.0,NaN
Afghanistan,2020-12-08,48366.0,230.0,8425.0,1908.0,6.0
Afghanistan,2020-12-07,48136.0,235.0,8250.0,1902.0,27.0
Afghanistan,2020-12-06,47901.0,260.0,8244.0,1875.0,10.0
Afghanistan,2020-12-05,47641.0,253.0,8291.0,1865.0,18.0


In [62]:
# Example to get specific row
#world_cases.loc[idx["Norway", '2021-03-29'], :]

In [63]:
world_pop = pd.read_csv('data/world_population.csv',
        index_col=['country'],
        squeeze=True
        ).sort_index()

In [64]:
world_pop

country
Afghanistan        38928346
Albania             2877797
Algeria            43851044
American Samoa        55191
Andorra               77265
                     ...   
Wallis & Futuna       11239
Western Sahara       597339
Yemen              29825964
Zambia             18383955
Zimbabwe           14862924
Name: population, Length: 235, dtype: int64

In [65]:
m_df = world_cases.merge(world_pop, left_index=True, right_index=True)

In [66]:
m_df

,date,cumulative_total_cases,daily_new_cases,active_cases,cumulative_total_deaths,daily_new_deaths,population
country,,,,,,,
Afghanistan,2020-02-15,0.0,NaN,0.0,0.0,NaN,38928346
Afghanistan,2020-12-08,48366.0,230.0,8425.0,1908.0,6.0,38928346
Afghanistan,2020-12-07,48136.0,235.0,8250.0,1902.0,27.0,38928346
Afghanistan,2020-12-06,47901.0,260.0,8244.0,1875.0,10.0,38928346
Afghanistan,2020-12-05,47641.0,253.0,8291.0,1865.0,18.0,38928346
...,...,...,...,...,...,...,...
Zimbabwe,2020-07-02,617.0,12.0,437.0,7.0,0.0,14862924
Zimbabwe,2020-07-01,605.0,14.0,432.0,7.0,0.0,14862924
Zimbabwe,2020-06-30,591.0,17.0,422.0,7.0,0.0,14862924


In [ ]:
# set to multiindex


In [76]:
world_measures = pd.read_csv('data/world_response_measures.csv',
        index_col=['country', 'date'],
        squeeze=True
        ).sort_index()

In [77]:
world_measures

,log_type,date
country,,
Afghanistan,Introduction / extension of measures,24/01/2020
Afghanistan,Introduction / extension of measures,07/03/2020
Afghanistan,Introduction / extension of measures,11/03/2020
Afghanistan,Introduction / extension of measures,12/03/2020
Afghanistan,Introduction / extension of measures,15/03/2020
...,...,...
Zimbabwe,Introduction / extension of measures,14/04/2020
Zimbabwe,Introduction / extension of measures,15/04/2020
Zimbabwe,Phase-out measure,19/04/2020
